**Problem 5b (Testing a sampler).** In this problem we will attempt to check whether the sampler we created in **Problem 2c** works correctly. To this end we will use a chi-squared goodness-of-fit test. This test works as follows:
 * Let $p_1,\ldots,p_d$ be the date frequencies as in the text file, scaled down to sum up to 1.
 * Use the sampler to generate a sample of dates. Let $c_1,\ldots,c_d$ be the observed counts, and let $f_i=Np_i$ be the expected counts, where $N$ is the sample size.
 * Compute the test statistic $$S = \sum_{i=1}^d \frac{\left(c_i-f_i\right)^2}{f_i}.$$
 * Our base assumption (the null hypothesis) $H_0$ is that our sampler works correctly. If $H_0$ is true AND if the expected count for each bucket is large enough, then $S$ has (approximately) a $\chi^2$ distribution with $d-1$ degrees of freedom.
 * Look up how likely is getting an $S$ value as large as the one you obtained if it has that distribution, i.e. the $p$-value. To do this use **scipy.stats.chi2.cdf**. If this value turns out smaller than the assumed threshold, e.g. $0.05$, we reject $H_0$. Otherwise we do not (we support $H_0$), but this does not mean $H_0$ is proved!
 * We mentioned earlier that expected counts for the buckets need to be large enough. "Large enough" assumption here is used to guarantee that $c_i$ are distributed approximately normally. Typically one requires that all counts are at least $5$. This is not the case in our problem (unless we take a huge sample) because of the errors in the data. The typical approach is to glue several buckets into one but this does not help in our case. Instead, ignore the erroneous dates when computing $c_i$ and $f_i$ and run the test again (on the same sample!). Remember to use a different number of degrees of freedom. Compare the results.
 * Perform the same test using **scipy.stats.chisquare** and compare the results.

In [11]:
import numpy as np
import scipy.stats

N = 2000
ITER = 10
########################################################################
def align(table):
    birth_all = np.sum(table)
    size = table.size
    extra = birth_all % size
    extra = size - extra
    table[:extra] += 1
    birth_all += extra
    table_height = birth_all/size
    return table_height

def transfer(table, barrier, donor, height):
    max_t = np.max(table)
    min_t = np.min(table)
    while max_t != min_t:
        transfer_amount = height - min_t
        index_max = np.argmax(table)
        index_min = np.argmin(table)
        table[index_max] -= transfer_amount
        table[index_min] += transfer_amount
        donor[index_min] = index_max
        barrier[index_min] = index_min + (min_t/height)
        max_t = np.max(table)
        min_t = np.min(table)
    for i in range(table.size):
        if barrier[i] == 0:
            barrier[i] = i + 1

def choose_day(barrier, donor):
    r = np.random.rand(N) * donor.size
    j = np.floor(r).astype('int64')
    #if r < barrier(floor(r)) return floor(r)
    #else return donor(floor(r))
    return np.concatenate((j[r < barrier[j]], donor[j[r >= barrier[j]]]), axis=None)
#########################################################################

Data = np.loadtxt('us_births_69_88.csv', skiprows=1, delimiter=',', dtype=np.int64)
table = Data[:,2]
error = np.where(table < 1000)
height = align(table)
donor = np.zeros(table.size, dtype=np.int64)
barrier = np.zeros(table.size, dtype=np.double)
transfer(table, barrier, donor, height)
########################################################################
def filter_data(data):
    #data[(tab < 1000)]
    return np.delete(data, error) #error is defined after reading us_births

def count_s(sample, exp):
    return np.sum(np.power(sample - exp, 2)/exp)

def print_res(s, sample, exp):
        print("Computed test statistic", s)
        print("chi2.cdf=", 1 - scipy.stats.chi2.cdf(s, sample.size - 1))
        print("chisquare=" , scipy.stats.chisquare(sample, f_exp=exp))

def results(iter_number):
    for i in range(iter_number):
        print(" *********** Iteration number ", i+1, "***********")
        print("Data without filter")
        exp = table/np.sum(table) * N #expected counts
        sample = choose_day(barrier, donor) #sampling with sampler from 2c
        sample = np.bincount(sample) #observed counts
        #add zeros to end if max(sample) < amount od days(last days aren't in sample)
        sample = np.pad(sample, (0, exp.size - sample.size), 'constant')
        s = count_s(sample, exp) #computation of the test statistic
        print_res(s, sample, exp)
        print("Filtered data")
        #ignoring the erroneous dates
        filter_exp = filter_data(exp)
        filter_sample = filter_data(sample)
        s = count_s(filter_sample, filter_exp)
        print_res(s, filter_sample, filter_exp)

results(ITER)



 *********** Iteration number  1 ***********
Data without filter
Computed test statistic 455.2
chi2.cdf= 0.0018183572479134602
chisquare= Power_divergenceResult(statistic=455.2, pvalue=0.0018183572479134778)
Filtered data
Computed test statistic 422.9419354838709
chi2.cdf= 0.01946676926922808
chisquare= Power_divergenceResult(statistic=422.9419354838709, pvalue=0.019466769269228114)
 *********** Iteration number  2 ***********
Data without filter
Computed test statistic 401.632
chi2.cdf= 0.13150398818511
chisquare= Power_divergenceResult(statistic=401.632, pvalue=0.13150398818511005)
Filtered data
Computed test statistic 369.3739354838709
chi2.cdf= 0.4262625317875909
chisquare= Power_divergenceResult(statistic=369.3739354838709, pvalue=0.42626253178759094)
 *********** Iteration number  3 ***********
Data without filter
Computed test statistic 400.51599999999996
chi2.cdf= 0.13997893369304282
chisquare= Power_divergenceResult(statistic=400.51599999999996, pvalue=0.1399789336930429)
Filt